References:

https://huggingface.co/docs/transformers/model_doc/bert
https://huggingface.co/docs/transformers/index
https://huggingface.co/docs/transformers/custom_datasets

In [1]:
!pip install transformers
!pip install wandb -qU

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [2]:
import wandb
wandb.login(key="aec6fef7ba56ee445129472eb583718b8e529934")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import pickle
import torch
import torch.nn as nn
from tqdm import tqdm
import transformers
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, BertTokenizerFast
from sklearn.utils.class_weight import compute_class_weight
import time
import copy
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score, recall_score



In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/mlops/reviews

/content/drive/.shortcut-targets-by-id/1pV4N9G9PWwlYVi6Fer0Tq0HFxZXbrdmI/reviews


In [7]:
def deflog(text):
    print(f"[DEF-LOG] {text}")

In [10]:

with open('train.pkl', 'rb') as f:
    train_set = pickle.load(f)
    
with open('test.pkl', 'rb') as f:
    test_set = pickle.load(f)

with open('val.pkl', 'rb') as f:
    val_set = pickle.load(f)         
 

df_train = pd.DataFrame.from_dict(train_set)
df_test =  pd.DataFrame.from_dict(test_set)
df_val =  pd.DataFrame.from_dict(val_set)
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()




# df_train = df_train.sample(frac=0.01, replace=True, random_state=1)

# df_test = df_test.sample(frac=0.01, replace=True, random_state=1)

# df_val = df_val.sample(frac=0.01, replace=True, random_state=1)



data = {'train': {}, "test": {}, 'val': {}}
data['train']['x'] = list(df_train['x'])
data['train']['y'] = list(df_train['y'])
data['test']['x'] = list(df_test['x'])
data['test']['y'] = list(df_test['y'])
data['val']['x'] = list(df_val['x'])
data['val']['y'] = list(df_val['y'])

_,data["train"]['x'],_,data["train"]['y'] = train_test_split(data["train"]['x'],data["train"]['y'],test_size=0.01,random_state=42,stratify=data["train"]['y'])

_,data["test"]['x'],_,data["test"]['y'] = train_test_split(data["test"]['x'],data["test"]['y'],test_size=0.01,random_state=42,stratify=data["test"]['y'])

_,data["val"]['x'],_,data["val"]['y'] = train_test_split(data["val"]['x'],data["val"]['y'],test_size=0.01,random_state=42,stratify=data["val"]['y'])


print("train size:",len(data["train"]['y'])) 
print("val size:",len(data["val"]['y']))
print("test size:", len(data["test"]['y']))
 
del df_train
del df_test
del df_val
del train_set
del test_set
del val_set


train size: 7688
val size: 855
test size: 950


In [11]:
%cd /content/drive/MyDrive/mlops/bert-cased-01

/content/drive/MyDrive/mlops/bert-cased-01


In [23]:
config = {
  "lr": 1e-4,
  "weight_decay": 1e-8,
  "epoch_num": 100,
  "batch_size": 512
}
tags = ["new_data", "10percent", "BERT-Pretrained"]
#wandb.init(settings=wandb.Settings(start_method="fork"))
run = wandb.init(entity='sentimovie', project="bert-cased-model-on-0.01-data__", config=config, tags=tags)

epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▂▅▆▇▇▇███
train_loss,█▇▆▅▅▄▃▃▂▂▂▁
val_acc,▁▆▇▇▇▇▇▇███▇
val_loss,█▇▆▅▅▄▃▃▂▂▁▁
epoch,11
lr,0.0001
train_acc,60.49692
train_loss,0.66232
val_acc,59.18129


In [13]:
#load model and tokenizer
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer = BertTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Deciding the padding length
If we take the padding length as the maximum length of text found in the training texts, it might leave the training data sparse. Taking the least length would in turn lead to loss of information. Hence, we would plot the graph and see the “average” length and set it as the padding length to trade-off between the two extremes

From the graph above, we take 17 as the padding length.

#Defining the model
We first freeze the BERT pre-trained model, and then add layers as shown in the following code snippets:


https://huggingface.co/docs/transformers/model_doc/bert

In [24]:
import random
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
set_seed(42) 

In [25]:
class BERT_architecture(nn.Module):

    def __init__(self, bert):
      
      super(BERT_architecture, self).__init__()
      


      self.bert = bert 
      
      for param in bert.parameters():
            param.requires_grad = False
      
      # dropout layer
      self.dropout = nn.Dropout(0.2)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x
     

In [26]:
class EarlyStopper:
    def __init__(self, patience, min_delta):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

class MyDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.targets = np.unique(targets, return_inverse=True)[1]
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
      return len(self.reviews)
  
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      padding="max_length",
      truncation=True,
      return_tensors='pt',
    )

    return [encoding['input_ids'].flatten(), encoding['attention_mask'].flatten()
            ,torch.tensor(target, dtype=torch.long)]
######
MAX_LEN = 170
def dataset_loader(data, config ,max_len,tokenizer):
    dataset = {'train': MyDataset(data['train']['x'], data['train']['y'],tokenizer=tokenizer,max_len=max_len),
              'val': MyDataset(data['val']['x'], data['val']['y'],tokenizer=tokenizer,max_len=max_len),
              'test': MyDataset(data['test']['x'], data['test']['y'],tokenizer=tokenizer,max_len=max_len)}
    
    deflog("Datasets created")

    dataloaders = {x: DataLoader(dataset[x], batch_size=config['batch_size'], 
                                 num_workers=1) for x in ['train', 'val', 'test']}
    deflog("Dataloaders created")
    dataset_sizes = {x: len(dataset[x]) for x in ['train', 'val', 'test']}
    deflog("Dataset_sizes created")
    return dataloaders, dataset, dataset_sizes
#####


In [27]:
def train_model(model, criterion, optimizer,  dataloaders, 
                dataset_sizes, device, config, earlyStopper, num_epochs=25):
    since = time.time()
    history = {'train_loss': [], 'val_loss': [], 
                    'train_acc': [], 'val_acc': []}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            total_loss, total_accuracy = 0, 0
            if phase == 'train':
                model.train()
                
                
            else:
                model.eval()

            total_loss = 0.0
            _accuracy = []
            running_corrects = 0
            for sent_id, mask, labels in tqdm(dataloaders[phase]):
                sent_id = sent_id.to(device)
                labels = labels.to(device)
                mask = mask.to(device)

                if phase == "train":
                  # clear previously calculated gradients 
                  model.zero_grad()
                  # get model predictions for the current batch
                  preds = model(sent_id, mask)
                else :
                  # deactivate autograd
                  with torch.no_grad():
                    preds = model(sent_id, mask)

                
                loss = criterion(preds, labels)
                total_loss +=   loss.item() * sent_id.size(0)
                if phase == "train":
                  loss.backward()
                  # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
                  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                  # update parameters
                  optimizer.step()

                else:
                  pass


                preds = torch.argmax(preds, dim=1).flatten()
                accuracy = (preds == labels).cpu().numpy().mean() * 100
                _accuracy.append(accuracy)

            avg_loss = total_loss / dataset_sizes[phase]
            # avg_loss = avg_loss / 10
            avg_accuracy = np.mean(_accuracy)

            deflog(f'{phase} Loss: {avg_loss:.4f} Acc: {avg_accuracy:.4f}')
            
  
            # predictions are in the form of (no. of batches, size of batch, no. of classes).

            history[f'{phase}_loss'].append(avg_loss)
            history[f'{phase}_acc'].append(avg_accuracy)
                    #######################################
            ###           wandb logging         ###
            #######################################
            ##        Log the training flow       ##
            #######################################
            tmp = {f'{phase}_loss': avg_loss, 
                   f'{phase}_acc': avg_accuracy,
                   'lr': config['lr'],
                   'epoch': epoch}
            wandb.log(tmp)
            #######################################
            #######################################

            if phase == 'val' and avg_accuracy > best_acc:
                best_acc = avg_accuracy
                torch.save(model.state_dict(), 'saved_weights.pt')
                best_model_wts = copy.deepcopy(model.state_dict())



        if earlyStopper.early_stop(history['val_loss'][-1]):
            deflog("Early Stopping")
            break  
    time_elapsed = time.time() - since
    deflog(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    deflog(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history


In [28]:
# dataset and dataloader
max_len = 170
dataloaders, dataset, dataset_sizes = dataset_loader(data, config,tokenizer=tokenizer,max_len=max_len)



# model
model = BERT_architecture(bert)
model.to(device)
#load weights of best model
# path = 'saved_weights.pt'
# model.load_state_dict(torch.load(path))
deflog(f"ML model defined (device: {device})")

earlyStopper = EarlyStopper(patience=10, min_delta=1e-2)
deflog("EarlyStopper defined")
####
#compute the class weights

class_weights = compute_class_weight('balanced', 
                                    classes=np.unique(data['train']['y']), 
                                    y=np.array(data['train']['y']))
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
deflog("Define loss class_weights")
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=wandb.config['lr'],
                        weight_decay=wandb.config['weight_decay'])
deflog("Optimizer (Adam) defined")

# cosin_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 
#                                                     T_max=config['epoch_num'], 
#                                                     eta_min=5e-6)
# deflog("Scheduler (cosin_lr_scheduler) defined")

model, history = train_model(model, criterion, optimizer,  
                              dataloaders, dataset_sizes, device, config,
                              earlyStopper, config['epoch_num'])

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


[DEF-LOG] Datasets created
[DEF-LOG] Dataloaders created
[DEF-LOG] Dataset_sizes created
[DEF-LOG] ML model defined (device: cuda:0)
[DEF-LOG] EarlyStopper defined
[DEF-LOG] Define loss class_weights
[DEF-LOG] Optimizer (Adam) defined


100%|██████████| 16/16 [01:23<00:00,  5.22s/it]


[DEF-LOG] train Loss: 0.6919 Acc: 51.9531


100%|██████████| 2/2 [00:09<00:00,  4.65s/it]


...

[DEF-LOG] val Loss: 0.6032 Acc: 65.1273


100%|██████████| 16/16 [01:24<00:00,  5.26s/it]


[DEF-LOG] train Loss: 0.6099 Acc: 66.4429


100%|██████████| 2/2 [00:09<00:00,  4.73s/it]

[DEF-LOG] val Loss: 0.6045 Acc: 64.7848
[DEF-LOG] Training complete in 156m 23s
[DEF-LOG] Best val Acc: 66.834172


In [29]:

torch.save(model.state_dict(), 'saved_weights.pt')
def save_results(model, history, base_path="output"):
    history_file_path = f"{base_path}/history.pkl"
    with open(history_file_path, 'wb') as handle:
        pickle.dump(history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    deflog(f'history has been saved in \"{history_file_path}\"')

    model_file_path = f"{base_path}/model.pt"
    torch.save(model.state_dict(), model_file_path)
    deflog(f'model has been saved in \"{model_file_path}\"')
     


In [30]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [31]:


def testing(model, dataloader, device):
    y_pred = []
    y_true = []
    model.eval()
    with torch.no_grad():
        for sent_id, mask, labels in tqdm((dataloader)):
            test_accuracy = []
            sent_id = sent_id.to(device)
            labels = labels.to(device)
            mask = mask.to(device)
            logits = model(sent_id, mask)
            preds = torch.argmax(logits, dim=1).flatten()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return y_pred, y_true
     


In [32]:
from sklearn import metrics

In [33]:
y_pred, y_true = testing(model, dataloaders['test'], device)

f1 = f1_score(y_true, y_pred, average='macro')
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
AUC = metrics.auc(fpr, tpr)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
acc = accuracy_score(y_true, y_pred)
deflog("Metric calculated")
wandb.sklearn.plot_confusion_matrix(y_true, 
                                    y_pred, 
                                    ['NORM', 'TUM'])
deflog("Send confusion matrix")
tmp = {
    "test_acc": acc, 
    "test_f1": f1, 
    "test_auc": AUC, 
    "test_precision": precision, 
    "test_recall": recall
}
wandb.log(tmp)
deflog("Run finished")

100%|██████████| 2/2 [00:10<00:00,  5.46s/it]


[DEF-LOG] Metric calculated
[DEF-LOG] Send confusion matrix
[DEF-LOG] Run finished


In [35]:
AUC
print("accuracy on test :",AUC)

accuracy on test : 0.6799159356725146
